# Import packages

In [2]:
import networkx as nx
import cobra 
import os
import pandas as pd
import sys
sys.path.append('../code/')
from graph_currency_metabolites_function import *

# Input and output files

In [3]:
pairs_of_currency_metabolites_file = "../data/external/pairs_of_currency_metabolites.csv" #the file stores the currency metabolite pairs
special_currency_metabolites_file = "../data/external/special_currency_metabolites.csv" #the file stores the special currency metabolites
use_model_file="../data/external/iML1515.xml" #the models
result_save_path = '../data/result/' # result folders

# Get currency metabolite from csv file

In [4]:
pairs_of_currency_metabolites = pd.read_csv(pairs_of_currency_metabolites_file)
special_currency_metabolites = pd.read_csv(special_currency_metabolites_file)

pi_pairs1 = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['pi_pairs1'] if str(x) != 'nan']
pi_pairs2 = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['pi_pairs2'] if str(x) != 'nan']
h_pairs1 = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['h_pairs1'] if str(x) != 'nan']
h_pairs2 = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['h_pairs2'] if str(x) != 'nan']
nh4_pairs = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['nh4_pairs'] if str(x) != 'nan']
other_pairs = [tuple(x.split(",")) for x in pairs_of_currency_metabolites['other_pairs'] if str(x) != 'nan']

excluded = [x for x in special_currency_metabolites['excluded'] if str(x) != 'nan']
nocarbon = [x for x in special_currency_metabolites['no carbon metabolites'] if str(x) != 'nan']
ions = [x for x in special_currency_metabolites['ions'] if str(x) != 'nan']
currency_mets = excluded+ nocarbon+ ions

# Get graph theory model 

In [5]:
model_name = use_model_file.split('/')[-1].split('.')[0]
print(model_name)
resultfile=result_save_path+ model_name+'_metabolite_links.txt'
model = cobra.io.read_sbml_model(use_model_file)

metall=[]
for met in model.metabolites:
    me = met.elements
    metid = met.id
    if 'C' in me and metid[:-2] != "co2" and metid[:-2] != "hco3" and metid[:-2] != "ACP":
        # print(metid)
        metall.append(metid)

G=nx.MultiDiGraph()
for rea in model.reactions:
    if not rea.boundary and 'BIOMASS' not in rea.id: 
        sub_pro = get_metpair(rea,pi_pairs1,h_pairs1,pi_pairs2,h_pairs2,nh4_pairs,other_pairs,currency_mets) 
        for sp in sub_pro:
            G.add_edge(sp[0], sp[1], label=rea.id)
            if rea.reversibility:
                G.add_edge(sp[1], sp[0], label=rea.id)

nx.write_edgelist(G, resultfile)
print('finish')

iML1515
finish
